In [1]:
from datasets import load_dataset
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
import sentencepiece as spm
from torch.nn.utils.rnn import pad_sequence

c:\Users\18327\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = load_dataset("spider")

Found cached dataset spider (C:/Users/18327/.cache/huggingface/datasets/spider/spider/1.0.0/4e5143d825a3895451569c8b9b55432b91a4bc2d04d390376c950837f4680daa)
100%|██████████| 2/2 [00:00<00:00, 21.56it/s]


In [3]:
# Data Sample
pd.DataFrame(data["train"]).head()

,db_id,query,question,query_toks,query_toks_no_value,question_toks
0,department_management,SELECT count(*) FROM head WHERE age > 56,How many heads of the departments are older th...,"[SELECT, count, (, *, ), FROM, head, WHERE, ag...","[select, count, (, *, ), from, head, where, ag...","[How, many, heads, of, the, departments, are, ..."
1,department_management,"SELECT name , born_state , age FROM head ORD...","List the name, born state and age of the heads...","[SELECT, name, ,, born_state, ,, age, FROM, he...","[select, name, ,, born_state, ,, age, from, he...","[List, the, name, ,, born, state, and, age, of..."
2,department_management,"SELECT creation , name , budget_in_billions ...","List the creation year, name and budget of eac...","[SELECT, creation, ,, name, ,, budget_in_billi...","[select, creation, ,, name, ,, budget_in_billi...","[List, the, creation, year, ,, name, and, budg..."
3,department_management,"SELECT max(budget_in_billions) , min(budget_i...",What are the maximum and minimum budget of the...,"[SELECT, max, (, budget_in_billions, ), ,, min...","[select, max, (, budget_in_billions, ), ,, min...","[What, are, the, maximum, and, minimum, budget..."
4,department_management,SELECT avg(num_employees) FROM department WHER...,What is the average number of employees of the...,"[SELECT, avg, (, num_employees, ), FROM, depar...","[select, avg, (, num_employees, ), from, depar...","[What, is, the, average, number, of, employees..."


In [4]:
train_df = pd.DataFrame(data["train"])[["query", "question"]]
val_df = pd.DataFrame(data["validation"])[["query", "question"]]

In [13]:
# pd.concat([question,query]).to_csv(r'sptrain.txt', header=None, index=None, sep=' ', mode='w')

vocab_size = 5000
# spm.SentencePieceTrainer.train(f'--input=sptrain.txt --model_prefix=m --vocab_size={vocab_size}')

In [14]:
# makes segmenter instance and loads the model file (m.model)
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [7]:
batch_size = 32 # Parallel Processing
block_size = 110 # Context
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [8]:
device

'cpu'

In [9]:
encode = lambda s: sp.encode(s, out_type=int, enable_sampling=True, alpha=0.1, nbest_size=-1,)
decode = lambda l: sp.decode(l)

In [15]:
vocab_size

5000

In [10]:
def get_batch(split):
    data =  train_df if split == "train" else val_df
    questions = data["question"].tolist()
    queries = data["query"].tolist()
    indices = torch.randperm(len(questions))[:batch_size] # TODO Is SPIDER dataset biased towards certain tasks? Then this is good way to randomize.
    x = pad_sequence([torch.tensor(encode(questions[i]), dtype=torch.long) for i in indices], batch_first=True, padding_value=0)
    y = pad_sequence([torch.tensor(encode(queries[i]), dtype=torch.long) for i in indices], batch_first=True, padding_value=0)
    x, y = x.to(device), y.to(device)
    return x, y

In [17]:
xb, xy = get_batch("train")

In [33]:
class SQLTModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets):
        
        logits = self.token_embedding_table(idx) # (B, T, C)
        
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        
        loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    

In [40]:
test = nn.Embedding(vocab_size, vocab_size)
test_logits = test(xb)
B, T, C = test_logits.shape
test_logits = test_logits.view(B*T, C)

In [41]:
test_logits

tensor([[-0.2297,  0.5606, -0.8360,  ...,  0.4177,  0.5404, -1.4870],
        [ 0.5601,  0.2138, -1.0444,  ...,  0.0552, -1.0280,  0.1359],
        [-0.4135,  0.1504, -0.8942,  ..., -1.3892, -0.1090, -0.6091],
        ...,
        [ 0.2016, -0.1102,  1.2851,  ...,  0.2378,  0.2525, -1.2562],
        [ 0.2016, -0.1102,  1.2851,  ...,  0.2378,  0.2525, -1.2562],
        [ 0.2016, -0.1102,  1.2851,  ...,  0.2378,  0.2525, -1.2562]],
       grad_fn=<ViewBackward0>)

In [34]:
model = SQLTModel(vocab_size)
out = model(xb, xy)
print(out.shape)
print(loss)

RuntimeError: shape '[2048]' is invalid for input of size 5984

In [29]:
nn.Embedding(vocab_size, vocab_size)

Embedding(5000, 5000)